<a href="https://colab.research.google.com/github/qjy9ey/qjy9ey/blob/main/python/superhero_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Superhero Database Assignment (SQLite & Google Colab)

## Objective
In this assignment, you will design and implement a **Superhero Database** using SQLite in Google Colab. This database will store superheroes, villains, and their battles. You will write SQL queries to analyze battle outcomes and relationships between heroes and villains.

### Group Size:
- 2 to 3 students per group.

### Tools Required:
- **Google Colab**
- **SQLite (built into Python)**
- **CSV files (provided for import)**

## Step 1: Setting Up Your SQLite Database

### Tasks:
1. Initialize SQLite in Google Colab.
2. Create a database named `superhero_db.sqlite`.
3. Define the database schema, ensuring it includes the following tables:
   - `heroes`: Stores superhero names, powers, and teams.
   - `villains`: Stores villains and their evil plans.
   - `battles`: Tracks battles between heroes and villains, including the outcome.
4. Implement primary keys and foreign keys where appropriate.

✅ **Checkpoint:** Ensure that the database and tables have been successfully created.

In [ ]:
import sqlite3

# Creates database.
connection = sqlite3.connect('superhero_db.sqlite')

print("Database created and connected!")

cursor = connection.cursor()

print("Cursor created!")

# Creates heroes table.
cursor.execute('''
CREATE TABLE IF NOT EXISTS heroes (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    power TEXT NOT NULL,
    team TEXT NOT NULL
)
''')

print("Heroes table created!")

# Creates villains table.
cursor.execute('''
CREATE TABLE IF NOT EXISTS villains (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    evil_plan TEXT NOT NULL
)
''')

print("Villains table created!")

# Creates battles table.
cursor.execute('''
CREATE TABLE IF NOT EXISTS battles (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    hero_id INTEGER,
    villian_id INTEGER,
    outcome TEXT NOT NULL
)
''')

print("Battles table created!")

## Step 2: Import Data from CSV Files

### Tasks:
1. Download the provided CSV files.
2. Upload them to Google Colab.
3. Write a script to import data from these files into the appropriate tables.
4. Verify that the data has been successfully inserted.

✅ **Checkpoint:** Ensure that each table contains the expected data from the CSV files.

In [ ]:
import pandas as pd

# Reads the data fom the heroes.csv file.
df_heroes = pd.read_csv('heroes.csv')

print(df_heroes.head())

# Puts data into heroes database.
df_heroes.to_sql('heroes', connection, if_exists='replace', index=False)

print("Data from CSV has been inserted into the heroes table.")

cursor.execute("SELECT * FROM heroes")
rows = cursor.fetchall()

print("Heroes data:")
for row in rows:
    print(row)

# Reads data from the villains.csv file.
df_villains = pd.read_csv('villains.csv')

df_villains.to_sql('villains', connection, if_exists='replace', index=False)

print("Data from CSV has been inserted into the villains table.")

cursor.execute("SELECT * FROM villains")
rows = cursor.fetchall()

print("Villains data:")
for row in rows:
    print(row)

# Reads data from the battles.csv file.
df_battles = pd.read_csv('battles.csv')

df_battles.to_sql('battles', connection, if_exists='replace', index=False)

print("Data from CSV has been inserted into the battles table.")

cursor.execute("SELECT * FROM battles")
rows = cursor.fetchall()

print("Battles data:")
for row in rows:
    print(row)

## Step 3: Querying the Database

### Tasks:
Write SQL queries to answer the following:
1. Retrieve all superheroes and their powers.
2. Find all battles where the hero won.
3. List all villains and their evil plans.
4. Determine which superhero has fought the most battles.

✅ **Checkpoint:** Ensure that all queries return meaningful results from the database.

In [ ]:
print("Retrieve all superheroes and their powers.")

# Retrieves the superhero name and power from the heroes database.
cursor.execute("SELECT name, power FROM heroes")

# Fetchall retrieves all the data.
results = cursor.fetchall()

for row in results:
  print(row)

print("Find all battles where the hero won.")
cursor.execute("SELET id, outcome FROM battles WHERE outcome = 'Hero Wins'")

# Only pulls the id when the outcome is that the Hero Wins.
results = cursor.fetchall()

for row in results:
  print(row)

print("List all villians and their evil plans.")

# Retrieves name and evil plans from villains database.
cursor.execute("SELECT name, evil_plan FROM villains")
results = cursor.fetchall()

for row in results:
  print(row)

# Determine which superhero has fought the most battles.
print("Determine which superhero has fought the most battles.")
cursor.execute("""
  SELECT heroes.name, COUNT(battles.hero_id) AS battle_count
  FROM battles
  JOIN heroes ON battles.hero_id = heroes.id
  GROUP BY battles.hero_id
  ORDER BY battle_count DESC
  LIMIT 1
""")

# Fetches the first one which would be the hero with the most battles.
if result:
  print(f"Superhero with the most battles: {result[0]}, Battles fought:
  {result[1]}")
else:
        print("No battles found in the database.")

## Step 4: Add a Custom Feature

### Tasks:
1. Design and implement an additional table related to superheroes or villains.
2. Populate it with data.
3. Write at least one query to retrieve meaningful insights from the new table.

**Examples of Custom Features:**
- Sidekicks for superheroes
- Villain hideouts
- Battle locations
- Superhero equipment

✅ **Checkpoint:** Ensure that the new table and queries function correctly.

In [ ]:
# Create the sidekicks table that includes name of the sidekick, which superhero they are the sidekick of, and their powers.
connection.execute('''
CREATE TABLE IF NOT EXISTS sidekicks(
  id INTEGER PRIMARY KEY AUTOINCREMENT
  name TEXT NOT NULL,
  hero_id INTEGER NOT NULL,
  power TEXT,
  FOREIGN KEY(hero_id) REFERENCES heroes(id)
)
''')

print("Sidekicks table created!")

# Insert sample data into the sidekicks table.
sidekick_data = [
    ('Ned', 1, 'Tech Support'),
    ('War Machine', 2, 'Armored Suit'),
    ('Robin', 3, 'Martial Arts'),
    ('Jimmy Olsen', 4, 'Friend'),
    ('Miek', 5, 'N/A'),
    ('Cassie Sandsmark', 6, 'Wonder Girl'),
    ('Shuri', 7, 'Gadgets'),
    ('Wally West', 8, 'Kid Flash'),
    ('Rick Jones', 9, 'A-Bomb'),
    ('Wong', 10, 'Sorceror')
]

# Insert the data into the database.
cursor.executemany('''
INSERT INTO sidekicks (name, hero_id, power)
VALUES (?, ?, ?)
''', sidekick_data)

connection.commit()
print("Sample sidekicks inserted!")

# Find all superheroes along with their sidekicks and their powers.
query_hero_sidekicks = '''
SELECT heroes.name AS hero_name, sidekicks.name AS sidekick_name, sidekicks.power
AS sidekick_power
FROM sidekicks
JOIN heroes ON sidekicks.hero_id = heroes.id
ORDER BY heroes.name;
'''

df_hero_sidekicks = pd.read_sql_query(query_hero_sidekicks, connection)
print("Superheroes and their sidekicks:")
print(df_hero_sidekicks)

## Bonus Challenge (+5 Points)

### Tasks:
1. Enhance your project by adding a visual or interactive component.

**Options:**
- Visualize battles using Matplotlib (e.g., a bar chart showing how many battles each superhero has fought).
- Make it interactive (e.g., allow users to input a superhero name and display their battles).

✅ **Checkpoint:** If attempting the bonus, ensure that your visualization or interactive component works correctly.

## Submission Guidelines

Each group must submit a **Google Colab Notebook (.ipynb)** containing:
- ✅ **Database schema (DDL)**
- ✅ **CSV imports & data insertion (DML)**
- ✅ **SQL queries & results**
- ✅ **Custom feature (bonus if applicable)**

In [ ]:
import pandas as pd

# Import matplotlib to create a bar chart.
import matplotlib.pyplot as plt

# Count the number of battles per superhero.
query_battles_count = '''
SELECT heroes.name AS hero_name, COUNT(battles.id) AS battle_count FROM battles
JOIN heroes ON battles.hero_id = heroes.id
GROUP BY heroes.name
'''

# Load results into a dataframe.
df_battles_count = pd.read_sql_query(query_battles_count, connection)

# Plot the bar chart.
plt.bar(df_battles_count['hero_name'], df_battles_count['battle_count'],color='blue')
plt.title('Number of Battles per Superhero')
plt.xlabel('Superhero')
plt.ylabel('Battle Count')
plt.xticks(rotation = 45)
plt.show()

## Grading Criteria (Total: 25 Points)

| Section                 | Points |
|-------------------------|--------|
| Database Setup          | 5      |
| Data Importing         | 10     |
| Querying the Data      | 10     |
| Extra Feature          | 5      |
| Bonus Challenge (Optional) | +5  |

### Final Notes:
- Be **creative** when designing superheroes and villains.
- **Comment your code** to explain its functionality.
- **Have fun** building your superhero universe!